## Decision Tree Algorithm Implements(roughly)

### Loss functions

#### Entropy
$Entropy(A) = -\sum^{m}_{k=1} p_k log_2(p_k)$ <br>
**ex)** Red balls : 10, Blue balls : 6,  $p_1=\frac{10}{16}, p2=\frac{6}{16}$<br>
if classified (7,1),(3,5), then $Entropy(A) = \sum^{d}_{i=1}R_i(-\sum^{m}_{k=1}p_k log_2(p_k)) (when \ R_i = 1/n)$<br>

#### Gini Index
$G.I(A) = \sum^{d}_{i=1}(R_i(1-\sum^{m}_{k=1}p^2_{ik}))$

We will use **Entropy function** as loss function

### Model Learning Process
> 1. Finding column and row condition subject to minimize entropy(loss function)
2. Split data frame by condition
3. Repeat recursivly until can't reducing entropy or entropy=0 

In [4]:
import pandas as pd
import numpy as np

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [29]:
#entropy function
def entropy(arr):
    if(len(arr)==0):
        return(0)
    
    p1 = sum(arr)/len(arr)
    p2 = (len(arr)-sum(arr))/len(arr)
    if((p1==0)|(p2==0)):
        return(0)
    
    result = -(p1*np.log2(p1) + p2*np.log2(p2))
    
    return(result)

In [30]:
def entropy_multi(arrs):
    entropy_value = 0
    n = sum([len(x) for x in arrs])
    for x in arrs:
        entropy_value += len(x)*entropy(x)/n
    return(entropy_value)

In [31]:
#Node is decision tree object
class Node:
    def __init__(self, df):
        self.df = df
        self.col = None
        self.row = None
        self.entropy = None
        self.left = None
        self.right = None

In [32]:
#Find minimizing entropy conditions(col, row)
def splitDf(df):
    minimum=1
    colnames = df.columns
    
    colRow = [colnames[0],df[colnames[0]].unique()[0]] #default value
    for col in colnames[:-1]:

        for row in df[col].unique():
            ys = []
            ys = ys+[df[df[col]==row]['value'].values,df[df[col]!=row]['value'].values]
            if(entropy_multi(ys)<minimum):
                minimum = entropy_multi(ys)
                colRow = [col,row]
                
    return(colRow,minimum)

In [33]:
#Make tree recursivly
def getNodes(node, df,entropyBefore = -1):
    if(entropy(df['value'])==0):
        node.entropy = 0
        return 0
    if(entropy(df['value'])==entropyBefore):
        return 0
    
    cr = splitDf(df)[0]
    node.entropy = entropy(df['value'])
    
    dfLeft = df[df[cr[0]]==cr[1]]
    dfRight = df[df[cr[0]] != cr[1]]
    
    node.col = cr[0]
    node.row = cr[1]
    
    node.left = Node(dfLeft)
    node.right = Node(dfRight)
    
    getNodes(node.left, dfLeft,node.entropy)
    getNodes(node.right, dfRight, node.entropy)

In [34]:
#Check decision tree
def treePrint(node,treeList,right):
    
    
    treeList_ = treeList.copy()
    
    if(right =='start'):
        0
    elif(right):
        treeList_.append("right")
    else:
        treeList_.append("left")
    
    
    if((node.left ==None)&(node.right ==None)):
        print(treeList_)
        return 0
    treeList_.append((node.col,node.row))

    
        
   
    treePrint(node.left, treeList_,0)
    treePrint(node.right,treeList_,1)

In [35]:
#Example data
inputs = [
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'no'}, False),
    ({'level': 'Senior', 'lang': 'Java', 'tweets': 'no', 'phd': 'yes'}, False),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, False),
    ({'level': 'Mid', 'lang': 'R', 'tweets': 'yes', 'phd': 'yes'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'no', 'phd': 'no'}, False),
    ({'level': 'Senior', 'lang': 'R', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Senior', 'lang': 'Python', 'tweets': 'yes', 'phd': 'yes'}, True),
    ({'level': 'Mid', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, True),
    ({'level': 'Mid', 'lang': 'Java', 'tweets': 'yes', 'phd': 'no'}, True),
    ({'level': 'Junior', 'lang': 'Python', 'tweets': 'no', 'phd': 'yes'}, False)
]

X = pd.DataFrame([x[0] for x in inputs])

Y = pd.DataFrame(x[1] for x in inputs)

Y.columns = ['value']

df = pd.concat([X,Y],axis=1)

arr = Y.value.values

In [36]:
root=Node(df)

getNodes(root,df)

In [37]:

treePrint(root,[],'start')
'''
left is true and right is false
check this
'''
#root.left.df

[('level', 'Mid'), 'left']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'left', ('level', 'Senior'), 'left']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'left', ('level', 'Senior'), 'right', ('phd', 'no'), 'left']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'left', ('level', 'Senior'), 'right', ('phd', 'no'), 'right']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'right', ('phd', 'no'), 'left']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'right', ('phd', 'no'), 'right', ('level', 'Junior'), 'left']
[('level', 'Mid'), 'right', ('tweets', 'no'), 'right', ('phd', 'no'), 'right', ('level', 'Junior'), 'right']


'\nleft is true and right is false\ncheck this\n'

In [38]:
#Prediction by X
def dfPred(node, sr):

    if((node.left==None)&(node.right==None)):
        if(len(node.df['value']) != 0):
            result[0] = int(round(node.df['value'].sum()/len(node.df['value']),0))
            return 0
        else:
            result[0] =  np.random.randint(2) #0,1 is 50%, 50%
            return 0

    if(sr[node.col] == node.row):
        dfPred(node.left, sr)
    else:
        dfPred(node.right, sr)

In [39]:
#labels is predict label
labels = []

for i in range(len(df)):
    result=[True]
    dfPred(root, df.loc[i])
    labels = labels+result

### My decision tree vs sklearn decision tree

In [40]:
import time

In [41]:
dataset = pd.read_csv("titanic.csv")
dataset.head()
#dataset=dataset.rename(columns = {"Outcome":"value"})

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [42]:
#select discrete values by X
dataset=dataset[['Pclass','Sex','SibSp','Parch','Embarked','Survived']]
#y label's column name must be 'value'
dataset=dataset.rename(columns = {"Survived":"value"})

In [43]:
#set seed
np.random.seed(2020)
#shuffle
dataset = dataset.sample(frac=1).reset_index(drop=True)

train = dataset[:100]
test = dataset[-100:]

In [44]:
#accuracy function
def accuracy(real,pred):
    return(1-sum(abs(real-pred))/len(real))

In [45]:
start = time.time()
root=Node(train)

getNodes(root,train)
print(time.time() - start)

0.3237636089324951


In [46]:
labels = []

for i in range(len(test)):
    result=[True]
    dfPred(root, test.iloc[i])
    labels = labels+result

In [47]:
accuracy(test['value'],labels)

0.78

In [48]:
train_ = train.apply(lambda col: pd.factorize(col, sort=True)[0])
test_ = test.apply(lambda col: pd.factorize(col, sort=True)[0])

In [49]:
start = time.time()
dtc = DecisionTreeClassifier(criterion="entropy")
dtc = dtc.fit(train_.iloc[:,:-1],train_.iloc[:,-1])

pred = dtc.predict(test_.iloc[:,:-1])
print(time.time()-start)


0.0030803680419921875


In [50]:
accuracy(test['value'],pred)

0.77

### Result

- My algorithm's running time is very slower than sklearn's (100 times slower)<br>
- Result is similar (78% vs 77%)<br>
We did recursive partitioning and it can induce 'over fitting'. So 'pruning process' is required.<br>
$CC(T) = Err(T) + \alpha \times L(T)$ can be used for pruning